# deepspeech library

In [ ]:
# we can use this command to get the transcript of an audio file
!deepspeech --model models/output_graph.pbmm --alphabet models/alphabet.txt --lm models/lm.binary --trie models/trie --audio /home/aloui/Desktop/eurecom/semesterproject/pretrained_model/deepspeech_pytorchv2/data/LibriSpeech_dataset/test_clean/wav/61-70968-0002.wav


Loading model from file models/output_graph.pbmm
TensorFlow: v1.12.0-10-ge232881
DeepSpeech: v0.4.1-0-g0e40db6
2019-04-30 16:46:51.922214: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loaded model in 0.0748s.
Loading language model from files models/lm.binary models/trie


# evaluate the two model deepspeech and the google api using the librispeech dataset

In [2]:
from __future__ import absolute_import, division, print_function

import argparse
import numpy as np
import shlex
import subprocess
import sys
import wave

from deepspeech import Model, printVersions
from timeit import default_timer as timer

try:
    from shhlex import quote
except ImportError:
    from pipes import quote

In [3]:


# These constants control the beam search decoder

# Beam width used in the CTC decoder when building candidate transcriptions
BEAM_WIDTH = 50

# The alpha hyperparameter of the CTC decoder. Language Model weight
LM_ALPHA = 0.75

# The beta hyperparameter of the CTC decoder. Word insertion bonus.
LM_BETA = 1.85


# These constants are tied to the shape of the graph used (changing them changes
# the geometry of the first layer), so make sure you use the same constants that
# were used during training

# Number of MFCC features to use
N_FEATURES = 26

# Size of the context window used for producing timesteps in the input vector
N_CONTEXT = 9

def convert_samplerate(audio_path):
    sox_cmd = 'sox {} --type raw --bits 16 --channels 1 --rate 16000 --encoding signed-integer --endian little --compression 0.0 --no-dither - '.format(quote(audio_path))
    try:
        output = subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
    except OSError as e:
        raise OSError(e.errno, 'SoX not found, use 16kHz files or install it: {}'.format(e.strerror))

    return 16000, np.frombuffer(output, np.int16)

model='/home/aloui/Desktop/eurecom/semesterproject/DeepSpeech/models/output_graph.pbmm'
lm='/home/aloui/Desktop/eurecom/semesterproject/DeepSpeech/models/lm.binary'
trie='/home/aloui/Desktop/eurecom/semesterproject/DeepSpeech/models/trie'
alphabet='/home/aloui/Desktop/eurecom/semesterproject/DeepSpeech/models/alphabet.txt'

print('Loading model from file {}'.format(model), file=sys.stderr)
model_load_start = timer()
ds = Model(model, N_FEATURES, N_CONTEXT, alphabet, BEAM_WIDTH)
model_load_end = timer() - model_load_start
print('Loaded model in {:.3}s.'.format(model_load_end), file=sys.stderr)

if lm and trie:
    print('Loading language model from files {} {}'.format(lm, trie), file=sys.stderr)
    lm_load_start = timer()
    ds.enableDecoderWithLM(alphabet, lm, trie, LM_ALPHA, LM_BETA)
    lm_load_end = timer() - lm_load_start
    print('Loaded language model in {:.3}s.'.format(lm_load_end), file=sys.stderr)


Loading model from file /home/aloui/Desktop/eurecom/semesterproject/DeepSpeech/models/output_graph.pbmm
Loaded model in 0.129s.
Loading language model from files /home/aloui/Desktop/eurecom/semesterproject/DeepSpeech/models/lm.binary /home/aloui/Desktop/eurecom/semesterproject/DeepSpeech/models/trie
Loaded language model in 39.6s.


In [ ]:
#trying 

In [4]:
from os import listdir
from os.path import join
onlyfiles = [f[:len(f)-3] for f in listdir("/home/aloui/Desktop/eurecom/semesterproject/pretrained_model/deepspeech_pytorchv2/data/LibriSpeech_dataset/test_clean/test")]

In [16]:
print('Running inference.', file=sys.stderr)
inference_start = timer()
audio_path='/home/aloui/Desktop/eurecom/semesterproject/pretrained_model/deepspeech_pytorchv2/data/LibriSpeech_dataset/test_clean/wav/61-70968-0021.wav'
fin = wave.open(audio_path, 'rb')
fs = fin.getframerate()
if fs != 16000:
    print('Warning: original sample rate ({}) is different than 16kHz. Resampling might produce erratic speech recognition.'.format(fs), file=sys.stderr)
    fs, audio = convert_samplerate(audio)
else:
    audio = np.frombuffer(fin.readframes(fin.getnframes()), np.int16)

audio_length = fin.getnframes() * (1/16000)
fin.close()

print(ds.stt(audio, fs))
inference_end = timer() - inference_start
print('Inference took %0.3fs for %0.3fs audio file.' % (inference_end, audio_length), file=sys.stderr)


Running inference.


surely we can submit with good grace


Inference took 14.414s for 2.690s audio file.


In [17]:
import IPython
IPython.display.Audio(audio_path)

In [68]:

test=''
i=0
ground_truth=''
for file in set(onlyfiles):
    lines = open("/home/aloui/Desktop/eurecom/semesterproject/pretrained_model/deepspeech_pytorchv2/data/LibriSpeech_dataset/test_clean/test/"+file+"txt").readlines()
    #print(lines)
    ground_truth=ground_truth+lines[0]
    audio="/home/aloui/Desktop/eurecom/semesterproject/pretrained_model/deepspeech_pytorchv2/data/LibriSpeech_dataset/test_clean/test/"+file+"wav"
    fin = wave.open(audio, 'rb')
    fs = fin.getframerate()
    if fs != 16000:
        print('Warning: original sample rate ({}) is different than 16kHz. Resampling might produce erratic speech recognition.'.format(fs), file=sys.stderr)
        fs, audio = convert_samplerate(audio)
    else:
        audio = np.frombuffer(fin.readframes(fin.getnframes()), np.int16)

    audio_length = fin.getnframes() * (1/16000)
    fin.close()
    if i%50==0:
        print(i)
    i+=1
    test=test+ds.stt(audio, fs)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [69]:
#word error rate for the deepspeech model
error = wer(ground_truth.lower(), test)
print('error %f'%error)

error 0.099790


# real time speech recognition

using library called speechrecognition

In [18]:
import random
import time

import speech_recognition as sr


def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.
    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio,language='fr-FR')
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response



In [19]:
import speech_recognition as sr

r = sr.Recognizer()
m = sr.Microphone()
print(recognize_speech_from_mic(r, m)['transcription'])

lalalala lalalala lalalala


# static speech recognition


In [34]:
import speech_recognition as sr

r = sr.Recognizer()

with sr.AudioFile('/home/aloui/Desktop/eurecom/semesterproject/DeepSpeech/file.wav') as source:
    audio = r.record(source)

r.recognize_google(audio, language='fr-FR')

'Bonjour Comment vas-tu je suis à Louis'

# record voice and save it as a file.wav

In [51]:
import pyaudio
import wave
 
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "file.wav"
 
audio = pyaudio.PyAudio()
 
# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
print ("recording...")
frames = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print ("finished recording")
 
 
# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

recording...
finished recording


In [3]:
!pip3 install --user jiwer


In [4]:
#testing the wer
from jiwer import wer

ground_truth = "hello world"
hypothesis = "hello word"

error = wer(ground_truth, hypothesis)
print(error)

0.5


In [19]:
ground_truth

'hello world'

In [36]:
from os import listdir
from os.path import join
onlyfiles = [f[:len(f)-3] for f in listdir("/home/aloui/Desktop/eurecom/semesterproject/pretrained_model/deepspeech_pytorchv2/data/LibriSpeech_dataset/test_clean/test")]

In [37]:
onlyfiles=[set(onlyfiles)]


In [39]:
len(onlyfiles[0])

999

In [40]:
%%time
import speech_recognition as sr

r = sr.Recognizer()
test=''
i=0
ground_truth=''
for file in set(onlyfiles[0]):
    lines = open("/home/aloui/Desktop/eurecom/semesterproject/pretrained_model/deepspeech_pytorchv2/data/LibriSpeech_dataset/test_clean/test/"+file+"txt").readlines()
    #print(lines)
    ground_truth=ground_truth+lines[0]
    with sr.AudioFile("/home/aloui/Desktop/eurecom/semesterproject/pretrained_model/deepspeech_pytorchv2/data/LibriSpeech_dataset/test_clean/test/"+file+"wav") as source:
        audio = r.record(source)
    if i%50==0:
        print(i)
    i+=1
    test=test+r.recognize_google(audio)
    


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
CPU times: user 9.74 s, sys: 21.5 s, total: 31.2 s
Wall time: 30min 48s


## len(test)

In [41]:
len(ground_truth)

2704

In [43]:
#word error rate of the google api
from jiwer import wer

error = wer(ground_truth.lower(), test)
print('error %f'%error)

error 0.190184
